<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_01_4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 1: Course Overview**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 1 Material

* Part 1.1: Course Overview [[Video]]() [[Notebook]](t81_558_class_01_1_overview.ipynb)
* Part 1.2: Generative AI Overview [[Video]]() [[Notebook]](t81_558_class_01_2_genai.ipynb)
* Part 1.3: Introduction to OpenAI [[Video]]() [[Notebook]](t81_558_class_01_3_openai.ipynb)
* **Part 1.4: Introduction to LangChain** [[Video]]() [[Notebook]](t81_558_class_01_4_langchain.ipynb)
* Part 1.5: Prompt Engineering [[Video]]() [[Notebook]](t81_558_class_01_5_prompt_engineering.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab


# Part 1.4: Introduction to LangChain

One of the most intriguing and promising developments in the evolving landscape of language models and artificial intelligence is LangChain. This technology represents a significant leap forward in how we interact with and harness the capabilities of large language models (LLMs). As we delve into the intricacies of LangChain in this chapter, it's important to understand not just the technical underpinnings but also the user experience that makes it so revolutionary.

## LangChain Chat Conversation Format

To explore LangChain comprehensively, we will adopt a format that has become increasingly familiar and effective in LLMs: the chat conversation interface. This interactive style, reminiscent of how many of us communicate daily, offers a unique and accessible means to illustrate LangChain's capabilities, potential applications, and the nuances of its operation.

We begin by importing the components from the LangChain library to support a chat-style interface to OpenAI. We will use the ChatOpenAI interface for the OpenAI family of LLM models.

In [2]:
# Conversation Style Inteface

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

The conversation format consists of arrays of chat entries of the following three types:

* **SystemMessage** - This class designates the system prompt that provides instructions to the AI on the nature of the conversation and hints and guidelines. Generally, there will be only one system message at the beginning of the array.
* **HumanMessage** - This class designates the chat messages from outside the LLM, typically the human user.
* **AIMessage** - This class designates the chat messages from the LLM as responses to the HumanMessage messages.

Here we see the chain to ask a simple question.

In [3]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that concisely and accurately."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]

We now submit these messages and retrieve the output from the model. We will use the older and less expensive GPT-3.5 model, which is good enough for this query. Further, we use a zero temperature; we are simply looking for a factual answer, and creativity is not a goal or concern.

In [4]:
MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  #model="gpt-4",
  temperature= 0.0,
  n= 1,
  max_tokens= 256)

print("Model response:")
output = llm.invoke(messages)
print(output.content)
print("-----------")
print(output.response_metadata)

Model response:
The capital of France is Paris.
-----------
{'token_usage': {'completion_tokens': 7, 'prompt_tokens': 30, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}


The model that LangChain returns to you returns additional metadata. This data shows the token usage, which might be useful for estimating the total cost expected from this query.

We can continue to grow this conversation if we wish. To do so, we added the model's response and another human question. Here, we will ask the model if it was sure about its last response.

In [5]:
messages.append(output)
messages.append(HumanMessage(content="Are you sure, I think it was renamed for some reason?"))
for message in messages:
    print(f"{type(message).__name__} : {message.content}")

SystemMessage : You are a helpful assistant that concisely and accurately.
HumanMessage : What is the capital of France?
AIMessage : The capital of France is Paris.
HumanMessage : Are you sure, I think it was renamed for some reason?


We can submit the conversation array to the model and see its latest response.

In [6]:
print("Model response:")
output = llm.invoke(messages)
print(output.content)

Model response:
Yes, I am sure. The capital of France is still Paris.


## Asking a Single Question

If you wish to ask the model a single question, not as part of a conversation chain, you can pass a string to the model for a response.

In [7]:
# complete

from langchain_openai import OpenAI, ChatOpenAI

MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = ChatOpenAI(model=MODEL, temperature=0)

# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the OpenAI API
# The method and parameters might differ based on the Langchain version
response = llm.invoke(question)

# Print the response
display(response.content)

'1. New York City, New York\n2. Los Angeles, California\n3. Chicago, Illinois\n4. Houston, Texas\n5. Phoenix, Arizona'

## Prompt Templates

LangChain allows you to create chains of operations typically performed as part of an LLM-enabled application. One of these operations is a prompt template, which allows you to insert text into a previously created prompt. In this example, we will create a prompt template that asks the model to create a random blog post title.

```
Return only the title of a blog post article title on the topic of {topic} in {language}
```

To accomplish this objective, we will use a **PromptTemplate** object.

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

topic = "pets for data scientists"
language = "english"

# Initialize the OpenAI LLM (Language Learning Model) with your API key
# Use higher temperature for greater creativity
llm = ChatOpenAI(model=MODEL, temperature=0.7)

title_template = PromptTemplate( input_variables = ['topic', 'language'],\
  template = 'Return only the title of a blog post article title on the topic of {topic} in {language}' )
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)
response = title_chain.run({'topic':topic,'language':language })
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Return only the title of a blog post article title on the topic of pets for data scientists in english

> Finished chain.
"Using Data Science to Improve Pet Health and Wellness"


## Create a Simple Sequential Chain

We will now use LangChain to tie multiple LLM calls into a longer chain using the **SimpleSequentialChain** class. We will use two smaller chains to create a title and body text for a blog post. We begin by defining the two prompts we will use to construct this blog post. Also, note that we request that the LLM utilize [markdown](https://en.wikipedia.org/wiki/Markdown) to generate the actual blog post.


In [9]:
# Create the two prompt templates
title_template = PromptTemplate( input_variables = ['topic'], template = 'Give me a blog post title on {topic} in English' )
article_template = PromptTemplate( input_variables = ['title'], template = 'Write a blog post for {title}, format in markdown.' )

We will create the first chain to generate the random title. Here, we allow the user to specify the topic. We use a higher temperature to increase the creativity of the title. We also use a simpler model to minimize cost for the relatively simple task of title selection.

In [10]:
MODEL = 'gpt-4o-mini'

# Create a chain to generate a random
llm = ChatOpenAI(model=MODEL, temperature=0.7)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)

Next, we compose the actual blog post; we will use a lower temperature to decrease creativity and cause the LLM to stick to factual information and avoid hallucinations. We also use a more complex model to provide a better article.

In [11]:
MODEL2 = 'gpt-4'

# Create the article chain
llm2 = ChatOpenAI(model=MODEL2, temperature=0.1)
article_chain = LLMChain(llm=llm2, prompt=article_template, verbose=True)

Now, we combine these two chains into one. The input to the first chain will be the selected topic. The first chain will then output the title to the second chain, which will, in turn, output the actual article.

In [12]:
# Create a complete chain to create a new blog post
complete_chain=SimpleSequentialChain(chains=[title_chain, article_chain], verbose=True)

We can now display the final article. In this case, we requested an article on "photography," and displayed the final article's markdown.

In [13]:
from IPython.display import display_markdown

article = complete_chain.invoke('photography')



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Give me a blog post title on photography in English

> Finished chain.
"Capturing Moments: The Art of Photography"


> Entering new LLMChain chain...
Prompt after formatting:
Write a blog post for "Capturing Moments: The Art of Photography", format in markdown.

> Finished chain.
# Capturing Moments: The Art of Photography

Photography is more than just clicking a button on a camera. It's an art form that captures moments, emotions, and stories in a single frame. It's about seeing the world from a different perspective and sharing that vision with others. In this blog post, we will delve into the art of photography and how it allows us to capture and preserve our most precious moments.

## The Power of a Photograph

A photograph has the power to evoke emotions, recall memories, and tell a story without uttering a single word. It's a snapshot of a moment in time, forever frozen fo

The actual display of the markdown is handled by this code:

In [14]:
output_key = complete_chain.output_key

display_markdown(article[output_key], raw=True)

# Capturing Moments: The Art of Photography

Photography is more than just clicking a button on a camera. It's an art form that captures moments, emotions, and stories in a single frame. It's about seeing the world from a different perspective and sharing that vision with others. In this blog post, we will delve into the art of photography and how it allows us to capture and preserve our most precious moments.

## The Power of a Photograph

A photograph has the power to evoke emotions, recall memories, and tell a story without uttering a single word. It's a snapshot of a moment in time, forever frozen for us to revisit whenever we please. Whether it's a breathtaking landscape, a candid moment of joy, or a poignant scene of hardship, a photograph can transport us to that moment, allowing us to experience the emotions and sensations associated with it.

## The Art of Capturing Moments

The art of photography lies in the ability to capture these moments. It's about more than just having the right equipment or knowing the technical aspects of how a camera works. It's about observing the world around you, understanding the interplay of light and shadow, and having the patience to wait for the perfect moment.

### Observation

Observation is a key skill in photography. It's about noticing the details that others might overlook - the way the light filters through the leaves, the expression on a person's face, or the patterns formed by a flock of birds in the sky. By paying attention to these details, you can capture a unique perspective that tells a story.

### Understanding Light

Light is one of the most important elements in photography. It can dramatically change the mood and atmosphere of a photograph. By understanding how light works and how to manipulate it, you can create stunning images that evoke a range of emotions.

### Patience

Patience is often overlooked in photography, but it's crucial for capturing the perfect moment. Sometimes, you need to wait for the right lighting conditions, for the perfect expression, or for the decisive moment when everything comes together. It's this patience that often separates a good photograph from a great one.

## The Joy of Photography

Photography is a rewarding hobby and profession. It allows us to express our creativity, capture our memories, and share our perspective with the world. Whether you're a professional photographer or a hobbyist, the joy of capturing a moment and preserving it in a photograph is a feeling that never gets old.

In conclusion, photography is a powerful art form that allows us to capture and preserve our most precious moments. It requires observation, understanding of light, and patience, but the rewards are well worth it. So, pick up your camera, start observing the world around you, and capture the moments that matter to you.